In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Dataset

/content/drive/MyDrive/Dataset


In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import pandas as pd
from PIL import Image

FOLDER_PATH_INP = "/content/drive/MyDrive/Dataset/Data_ver2_UTA-RLDD"
FOLDER_PATH_OUT = "/content/drive/MyDrive/Dataset/Data_out"
categories =  ["awake", "drowsy"]

In [4]:
path_for_face = "haarcascade_frontalface_default.xml"
path_for_eyes = "haarcascade_eye.xml"
path_for_nose = "haarcascade_mcs_nose.xml"
face_detector = cv2.CascadeClassifier(path_for_face)
eye_detector = cv2.CascadeClassifier(path_for_eyes)
nose_detector = cv2.CascadeClassifier(path_for_nose)

In [5]:
def euclidean_distance(a, b):
	x1 = a[0]; y1 = a[1]
	x2 = b[0]; y2 = b[1]
	return math.sqrt(((x2 - x1) * (x2 - x1)) + ((y2 - y1) * (y2 - y1)))

def detectFace(img):
	faces = face_detector.detectMultiScale(img, 1.1, 5)

	if len(faces) > 0:
		face = faces[0]
		face_x, face_y, face_w, face_h = face
		img = img[int(face_y):int(face_y+face_h), int(face_x):int(face_x+face_w)]
		img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
		
		return 1, img_gray
	else:
		img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
		return 0, img_gray
		#raise ValueError("No face found in the passed image ")

def alignFace(img_path):
    img = cv2.imread(img_path)

    img_raw = img.copy()

    stt, gray_img = detectFace(img)
 
    if stt == 0:
        return 0, img_raw

    eyes = eye_detector.detectMultiScale(gray_img)
    
    #print("found eyes: ",len(eyes))
    
    if len(eyes) >= 2:
        #find the largest 2 eye
        
        base_eyes = eyes[:, 2]
        #print(base_eyes)
        
        items = []
        for i in range(0, len(base_eyes)):
            item = (base_eyes[i], i)
            items.append(item)
        
        df = pd.DataFrame(items, columns = ["length", "idx"]).sort_values(by=['length'], ascending=False)
        
        eyes = eyes[df.idx.values[0:2]]
        
        #--------------------
        #decide left and right eye
        
        eye_1 = eyes[0]; eye_2 = eyes[1]
        
        if eye_1[0] < eye_2[0]:
            left_eye = eye_1
            right_eye = eye_2
        else:
            left_eye = eye_2
            right_eye = eye_1
        
        #--------------------
        #center of eyes
        
        left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
        left_eye_x = left_eye_center[0]; left_eye_y = left_eye_center[1]
        
        right_eye_center = (int(right_eye[0] + (right_eye[2]/2)), int(right_eye[1] + (right_eye[3]/2)))
        right_eye_x = right_eye_center[0]; right_eye_y = right_eye_center[1]
        
        #center_of_eyes = (int((left_eye_x+right_eye_x)/2), int((left_eye_y+right_eye_y)/2))
        
        cv2.circle(img, left_eye_center, 2, (255, 0, 0) , 2)
        cv2.circle(img, right_eye_center, 2, (255, 0, 0) , 2)
        #cv2.circle(img, center_of_eyes, 2, (255, 0, 0) , 2)
        
        #----------------------
        #find rotation direction
        
        if left_eye_y > right_eye_y:
            point_3rd = (right_eye_x, left_eye_y)
            direction = -1 #rotate same direction to clock
            # print("rotate to clock direction")
        else:
            point_3rd = (left_eye_x, right_eye_y)
            direction = 1 #rotate inverse direction of clock
            # print("rotate to inverse clock direction")
        
        #----------------------
        
        cv2.circle(img, point_3rd, 2, (255, 0, 0) , 2)
        
        cv2.line(img,right_eye_center, left_eye_center,(67,67,67),1)
        cv2.line(img,left_eye_center, point_3rd,(67,67,67),1)
        cv2.line(img,right_eye_center, point_3rd,(67,67,67),1)
        
        a = euclidean_distance(left_eye_center, point_3rd)
        b = euclidean_distance(right_eye_center, point_3rd)
        c = euclidean_distance(right_eye_center, left_eye_center)
        
        #print("left eye: ", left_eye_center)
        #print("right eye: ", right_eye_center)
        #print("additional point: ", point_3rd)
        #print("triangle lengths: ",a, b, c)
        
        cos_a = (b*b + c*c - a*a)/(2*b*c)
        #print("cos(a) = ", cos_a)
        angle = np.arccos(cos_a)
        #print("angle: ", angle," in radian")
        
        angle = (angle * 180) / math.pi
        # print("angle: ", angle," in degree")
        
        if direction == -1:
            angle = 90 - angle
        
        # print("angle: ", angle," in degree")
        
        #--------------------
        #rotate image
        
        new_img = Image.fromarray(img_raw)
        new_img = np.array(new_img.rotate(direction * angle))

        return 1, new_img

    return 1, img_raw


def crop_face(img_path):
    stt, alignedFace = alignFace(img_path)

    if stt == 0:
        return stt, alignedFace

    gray = cv2.cvtColor(alignedFace, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    if(len(faces) <= 0):
        return 2, alignedFace

    x, y, w, h = faces[0]
    # calculate the center of the face
    center_x = x + w // 2
    center_y = y + h // 2
    
    # calculate the size of the rectangle
    rect_size = min(w, h)
    # print(w, h, rect_size)
    
    # calculate the top-left corner of the rectangle
    rect_x = center_x - rect_size // 2
    rect_y = center_y - rect_size // 2
    
    # make sure the rectangle is within the bounds of the image
    rect_x = max(0, rect_x)
    rect_y = max(0, rect_y)
    rect_x2 = min(gray.shape[1], rect_x + rect_size)
    rect_y2 = min(gray.shape[0], rect_y + rect_size)
                
    rect_img = gray[rect_y:rect_y2, rect_x:rect_x2]
    ret = cv2.cvtColor(cv2.resize(rect_img, (72, 72)), cv2.COLOR_GRAY2RGB)

    return 1, ret

In [6]:
for categorie in categories:
    if not os.path.exists(f'{FOLDER_PATH_OUT}/{categorie}'):
        os.makedirs(f'{FOLDER_PATH_OUT}/{categorie}')

    for img_path in os.listdir(f'{FOLDER_PATH_INP}/{categorie}'):
        in_path = f'{FOLDER_PATH_INP}/{categorie}/{img_path}'
        out_name = f'{FOLDER_PATH_OUT}/{categorie}/{img_path}'

        # img = cv2.imread(in_path)
        # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        # plt.show()
        stt, ret = crop_face(in_path)
        if stt == 1:
            print(out_name)
            cv2.imwrite(out_name, ret)

            # plt.imshow(ret[:, :, ::-1])
            # plt.show()

/content/drive/MyDrive/Dataset/Data_out/awake/224101036.person05.awake.1d5d5598dd1b11ed8d5f2d5469c8be08.jpg
/content/drive/MyDrive/Dataset/Data_out/awake/224101036.person06.awake.629d657cdd1f11ed8d5f2d5469c8be08.jpg
/content/drive/MyDrive/Dataset/Data_out/awake/224101036.person06.awake.90eab160dd1e11ed8d5f2d5469c8be08.jpg
/content/drive/MyDrive/Dataset/Data_out/awake/224101036.person04.awake.07bea408dd1611ed8d5f2d5469c8be08.jpg
/content/drive/MyDrive/Dataset/Data_out/awake/224101036.person04.awake.defdc82cdd1611ed8d5f2d5469c8be08.jpg
/content/drive/MyDrive/Dataset/Data_out/awake/224101036.person06.awake.60f8b01add1e11ed8d5f2d5469c8be08.jpg
/content/drive/MyDrive/Dataset/Data_out/awake/224101036.person05.awake.7bce7a5edd1a11ed8d5f2d5469c8be08.jpg
/content/drive/MyDrive/Dataset/Data_out/awake/224101036.person04.awake.14cc61a2dd1711ed8d5f2d5469c8be08.jpg
/content/drive/MyDrive/Dataset/Data_out/awake/224101036.person06.awake.5c9fbc88dd1f11ed8d5f2d5469c8be08.jpg
/content/drive/MyDrive/Datas

ZeroDivisionError: ignored